In [1]:
import time
starttime=time.time()
print("Start time: {}".format(starttime))

Start time: 1554051211.416563


<h3>Please Note: </h3><br>
<p>There <b>are</b> errors in this file. I haven't done all the conversions of cardiovascular parameters, for eg. ml/hr to mcg/kg/min. Mainly because to do that, according to the internet, I need infusionrates and volumeoffluid which are NULL values for many records. First I need to clean them and then proceed. Also, keep in mind this is for the <b>demo</b> dataset. But, the same would work for the main dataset, as the tables are same.</p>
<p>Also, code optimization still needs to be done. Due to shortage of time, and the deadline for proposal submission coming soon, I might not be able to clean it up right away. Also, I would also like to make some statistics for this EDA.</p>
<p>At the end, you will notice that <b>t_sepsis</b> cases are less than the actual count of cases in the diagnoses table. This is simply because data needs to be cleaned and processed further. Also, this is a prediction metric and real life cases can be different. Once the coding season starts, I should be able to fine tune it to be as robust and accurate as possible.</p>

In [77]:
import pandas as pd
import math
import numpy as np
import collections

In [54]:
medication=pd.read_csv("medication.csv")

In [55]:
medication.sort_values(by=['drugorderoffset'],inplace=True)


In [56]:
microlab=pd.read_csv("microLab.csv")
microlab.head()

,microlabid,patientunitstayid,culturetakenoffset,culturesite,organism,antibiotic,sensitivitylevel
0,549559,983524,-688,"Blood, Venipuncture",gram positive rods,NaN,NaN
1,549555,983524,-5481,Stool,no growth,NaN,NaN
2,549565,983524,324,Stool,no growth,NaN,NaN
3,549562,983524,-686,"Blood, Central Line",gram negative rods,NaN,NaN
4,549564,983524,-681,"Blood, Central Line",gram positive cocci,NaN,NaN


In [57]:
microlab.sort_values(by=['culturetakenoffset'],inplace=True)

<h1>For t_suspicion</h1>

In [60]:
count=0
for val in medication['routeadmin']:
    if val=='IV':
        count+=1

In [61]:
count

21698

In [62]:
index=0
indices=[]
for val in medication['routeadmin']:
    if(val=="IV"):
        indices.append(index)
    index+=1 

we now have the indices where intervenous medication has been administered.
we have to check to t_suspicion for the given index. If doasge is more than 72 hours, then the start time is t_suspicion

In [63]:
index=0
medication_2=pd.DataFrame(medication,index=indices)

In [64]:
medication_2.head()

,medicationid,patientunitstayid,drugorderoffset,drugstartoffset,drugivadmixture,drugordercancelled,drugname,drughiclseqno,dosage,routeadmin,frequency,loadingdose,prn,drugstopoffset,gtc
0,7140253,142493,1024,1122,No,No,CLOPIDOGREL BISULFATE 75 MG PO TABS,NaN,75 3,PO,Daily,NaN,No,2903,0
8,9564809,142495,22,4,No,No,BISACODYL 10 MG RE SUPP,NaN,10 3,RE,Daily PRN,NaN,Yes,5963,0
9,10778912,142495,114,104,No,No,2 ML VIAL : ONDANSETRON HCL 4 MG/2ML IJ SOLN,NaN,4 3,IV,Q6H PRN,NaN,Yes,5963,0
11,11252618,142495,20,12,No,No,VITAMINS/MINERALS PO TABS,NaN,1 5002,PO,Daily,NaN,No,3728,0
12,8974838,142495,23,4,No,No,100 ML - MAGNESIUM SULFATE IN D5W 10-5 MG/ML...,NaN,1 4,IV,Daily PRN,NaN,Yes,5963,0


In [85]:
t_sus_culture={}
for pid in microlab['patientunitstayid'].unique():
    cultures=microlab.loc[microlab['patientunitstayid']==pid]
    cultures.sort_values(by=['culturetakenoffset'],inplace=True)
    earliest_culture=cultures['culturetakenoffset'].iloc[0]
    #print(earliest_culture)
    t_sus_culture[pid]=earliest_culture
    
            

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [86]:
t_sus_culture

{3155670: -438095,
 2586036: -78866,
 3150261: -45435,
 2596477: -35439,
 3136641: -32186,
 2597831: -28395,
 3138851: -22100,
 2600062: -22090,
 3050144: -21718,
 2590907: -20509,
 3150453: -20069,
 3041223: -19251,
 3138387: -18486,
 2622658: -17279,
 3149711: -16330,
 3132235: -16308,
 3133786: -16227,
 3152037: -15285,
 3114530: -15256,
 3051517: -15195,
 2613719: -14620,
 3158222: -14511,
 2634932: -14457,
 3104707: -13855,
 3147962: -13718,
 3158231: -13678,
 2576928: -13649,
 3155743: -13644,
 3135983: -13588,
 3157289: -13474,
 3069155: -13155,
 3143722: -12998,
 2583429: -12995,
 3145965: -12510,
 3153955: -12061,
 3143759: -12015,
 2595105: -11973,
 3136319: -11769,
 2609875: -11498,
 2612034: -11427,
 3140061: -11401,
 3141817: -11195,
 3139467: -10978,
 3137920: -10973,
 2590822: -10786,
 2634697: -10730,
 3132852: -10564,
 3154495: -10388,
 3143148: -10377,
 3131462: -10365,
 3137480: -10356,
 3135985: -10345,
 3046782: -10309,
 3140706: -10253,
 3051218: -10151,
 2609003:

In [69]:
med_antibiotics

,medicationid,patientunitstayid,drugorderoffset,drugstartoffset,drugivadmixture,drugordercancelled,drugname,drughiclseqno,dosage,routeadmin,frequency,loadingdose,prn,drugstopoffset,gtc
0,10741309,143599,14,436,Yes,No,CEFEPIME HCL 2 G IJ SOLR,10132.0,2000 MG,IV,Q24H,NaN,No,562,19
1,11276548,143599,78,1336,No,No,VANCOMYCIN 1.25 GM IN NS 250 ML IVPB (REPACKAGE),10093.0,"1,250 3",IV,Q24H,NaN,No,574,19
2,8010670,143603,11963,13480,No,No,VANCOMYCIN 1.25 GM IN NS 250 ML IVPB (REPACKAGE),10093.0,"1,250 3",IV,Daily,NaN,No,16525,19
3,10812292,153646,382,1296,No,No,VANCOMYCIN 1.25 GM IN NS 250 ML IVPB (REPACKAGE),10093.0,"1,250 3",IV,Daily,NaN,No,4070,19
4,11665407,165139,3,-592,No,No,VANCOMYCIN 1.25 GM IN NS 250 ML IVPB (REPACKAGE),10093.0,"1,250 3",IV,Q12H SCH,NaN,No,1435,19
5,8044270,171891,3,-48,No,No,VANCOMYCIN 1.25 GM IN NS 250 ML IVPB (REPACKAGE),10093.0,"1,250 3",IV,Q12H SCH,NaN,No,131,19
6,10453870,172015,12387,12473,No,No,LEVOFLOXACIN IN D5W 500 MG/100ML IV SOLN,12383.0,500 3,IV,Daily,NaN,No,13833,19
7,8369991,184757,15421,15462,No,No,VANCOMYCIN 1.25 GM IN NS 250 ML IVPB (REPACKAGE),10093.0,"1,250 3",IV,Daily,NaN,No,15649,19
8,8715649,201467,88,1281,Yes,No,CEFEPIME HCL 2 G IJ SOLR,10132.0,2000 MG,IV,Daily,NaN,No,1074,19
9,8946374,210208,59,715,No,No,VANCOMYCIN 1.25 GM IN NS 250 ML IVPB (REPACKAGE),10093.0,"1,250 3",IV,Q12H SCH,NaN,No,377,19


In [13]:
#From this result, I am assuming that for each patient in each table, offset starts from 0
medication_2.loc[medication['drugstartoffset']==2860]

#there are 5 records for the same drugstartoffset

,medicationid,patientunitstayid,drugorderoffset,drugstartoffset,drugivadmixture,drugordercancelled,drugname,drughiclseqno,dosage,routeadmin,frequency,loadingdose,prn,drugstopoffset,gtc
2,11626298,142495,2860,2860,No,No,NaN,NaN,NaN,IV,Once PRN,NaN,Yes,3056,0
2936,13007780,261520,2656,2860,Yes,No,NaN,8738.0,3.375 GM,IV,8H,NaN,No,2656,19
22318,13883122,296169,2867,2860,No,No,FUROSEMIDE INJ,3660.0,60 MG,IV,ONE,NaN,No,2861,56
34944,7421909,142495,2860,2860,No,No,NaN,NaN,5 3,IV,Once PRN,NaN,Yes,3056,0
40289,36359912,965080,2837,2860,No,No,NaN,2728.0,25 g,IV,q8hr (interval),NaN,No,4228,35


In [14]:
min_dosage_minutes=(72*60)

In [15]:
medication_2['diff_minutes']=medication_2.apply(lambda row: row.drugstopoffset-row.drugstartoffset, axis=1)

In [16]:
medication_2.head()

,medicationid,patientunitstayid,drugorderoffset,drugstartoffset,drugivadmixture,drugordercancelled,drugname,drughiclseqno,dosage,routeadmin,frequency,loadingdose,prn,drugstopoffset,gtc,diff_minutes
1,8428843,142495,220,207,No,No,1 ML - HYDRALAZINE HCL 20 MG/ML IJ SOLN,NaN,10 3,IV,Q6H PRN,NaN,Yes,5963,0,5756
2,11626298,142495,2860,2860,No,No,NaN,NaN,NaN,IV,Once PRN,NaN,Yes,3056,0,196
3,8109657,142495,220,204,No,No,2 ML - METOCLOPRAMIDE HCL 5 MG/ML IJ SOLN,NaN,5 3,IV,Q6H PRN,NaN,Yes,3728,0,3524
6,11547875,142495,24,4,No,No,MORPHINE INJ,NaN,2 3,IV,Q3H PRN,NaN,Yes,3370,0,3366
7,11640009,142495,23,4,No,No,MAGNESIUM SULFATE 2 G IN NS PREMIX,NaN,2 4,IV,Daily PRN,NaN,Yes,5963,0,5959


In [18]:
suspicious_intervals=[]
for diff in medication_2['diff_minutes']:
    if(diff>min_dosage_minutes):
        suspicious_intervals.append(diff)

In [19]:
len(suspicious_intervals)

4155

In [20]:
final_indices=medication_2.index[medication_2['diff_minutes']>min_dosage_minutes].tolist()

In [21]:
t_sus=[]
for f_ind in final_indices:
    t_sus.append((medication_2['patientunitstayid'][f_ind],medication_2['drugstartoffset'][f_ind],
                  medication_2['diff_minutes'][f_ind],medication_2['drugname'][f_ind]))

<h1>Here is the list of all the suspicious cases</h1>

<h3>Here is a list of all cases which have exceeded 72 hours of dosage, antibiotics included</h3>
<h4>Format is : (patientunitstay, drugstartoffset, total_time_dosage_given, drugname)</h4>

In [27]:
t_sus

[(142495, 207, 5756, '1 ML  -  HYDRALAZINE HCL 20 MG/ML IJ SOLN'),
 (142495, 4, 5959, 'MAGNESIUM SULFATE 2 G IN NS PREMIX'),
 (142495, 104, 5859, '2 ML VIAL : ONDANSETRON HCL 4 MG/2ML IJ SOLN'),
 (142495, 4, 5959, '100 ML  -  MAGNESIUM SULFATE IN D5W 10-5 MG/ML-% IV SOLN'),
 (145227, 3504, 4435, '1 ML  -  LORAZEPAM 2 MG/ML IJ SOLN'),
 (145227, 160, 7359, 'MORPHINE INJ'),
 (145227, 12, 7927, '2 ML VIAL : ONDANSETRON HCL 4 MG/2ML IJ SOLN'),
 (145917, 670, 6459, 'MAGNESIUM SULFATE 2 G IN NS PREMIX'),
 (145917, 77, 7052, '1 ML  -  HYDROMORPHONE HCL 1 MG/ML IJ SOLN'),
 (145917,
  670,
  6459,
  '100 ML  -  MAGNESIUM SULFATE IN D5W 10-5 MG/ML-% IV SOLN'),
 (146852,
  149,
  5868,
  '100 ML  -  MAGNESIUM SULFATE IN D5W 10-5 MG/ML-% IV SOLN'),
 (146852, 149, 5868, 'MAGNESIUM SULFATE 2 G IN NS PREMIX'),
 (149757, 1032, 4516, '4 ML  -  LABETALOL HCL 5 MG/ML IV SOLN'),
 (149757, 1032, 10238, 'MAGNESIUM SULFATE 2 G IN NS PREMIX'),
 (149757,
  1032,
  10238,
  '100 ML  -  MAGNESIUM SULFATE IN D5W 1

<h3>Antibiotics will be converted into a regex in future builds</h3>

In [23]:
#list of antibiotics screened out
final_tsus=[]
antibiotics_list=['adoxa','ala-tet','alodox','amikacin','amikin','amoxicillin','clavulanate','ampicillin',
                 'augmentin','avelox','avidoxy','azactam','azithromycin','aztreonam','axetil','bactocill',
                 'bactrim','bethkis','biaxin','bicillin l-a','cayston','cefazolin','cedax','cefoxitin',
                 'ceftazidime','cefaclor','cefadroxil','cefdinir','cefditoren','cefepime','cefotetan','cefotaxime',
                 'cefpodoxime','cefprozil','ceftibuten','ceftin','cefuroxime','cefuroxime','cephalexin','chloramphenicol',
                 'cipro','ciprofloxacin','claforan','clarithromycin','cleocin','clindamycin','cubicin','dicloxacillin',
                 'doryx','doxycycline','duricef','dynacin','ery-tab','eryped','eryc','erythrocin','erythromycin','factive',
                 'flagyl','fortaz','furadantin','garamycin','gentamicin','kanamycin','keflex','ketek','levaquin',
                 'levofloxacin','lincocin','macrobid','macrodantin','maxipime','mefoxin','metronidazole','minocin',
                 'minocycline','monodox','monurol','morgidox','moxatag','moxifloxacin','myrac','nafcillin sodium',
                 'nicazel doxy 30','nitrofurantoin','noroxin','ocudox','ofloxacin','omnicef','oracea','oraxyl','oxacillin',
                 'pc pen vk','pce dispertab','panixine','pediazole','penicillin','periostat','pfizerpen','piperacillin',
                 'tazobactam','primsol','proquin','raniclor','rifadin','rifampin','rocephin','smz-tmp','septra','septra ds',
                 'septra','solodyn','spectracef','streptomycin sulfate','sulfadiazine','sulfamethoxazole','trimethoprim',
                 'sulfatrim','sulfisoxazole','suprax','synercid','tazicef','tetracycline','timentin','tobi','tobramycin',
                 'trimethoprim','unasyn','vancocin','vancomycin','vantin','vibativ','vibra-tabs','vibramycin','zinacef',
                 'zithromax','zmax','zosyn','zyvox']
for i in t_sus:
    if(type(i[3]) is str):
        val=i[3].lower()
    if(any(subs in val for subs in antibiotics_list)):
        final_tsus.append(i)


In [28]:
tsus_dict={}
for row in final_tsus:
    if(row[0] in tsus_dict):
        if(tsus_dict[row[0]]>row[1]):
            tsus_dict[row[0]]=row[1]
        else:
            continue
    else:
        tsus_dict[row[0]]=row[1]
        

In [36]:
tsus_dict

{152954: 49,
 227279: 673,
 1057604: -3187,
 1062330: -138,
 143603: 1240,
 172764: 1231,
 181855: 76,
 188764: 5403,
 215156: 155,
 976781: -422,
 977887: 324,
 143599: 76,
 157644: 100368,
 179269: -348,
 184218: 84,
 186761: -787,
 203585: 1174,
 215535: 8471,
 215978: 147,
 233419: 1124,
 239049: 556,
 244216: 1021,
 249328: 1829,
 264276: 7976,
 272678: 15990,
 279513: 166,
 280568: 2106,
 287910: 616,
 305889: 8390,
 1057294: 9734,
 1072111: 11668,
 1117332: 72,
 151226: 533,
 151900: 395,
 172015: 26813,
 184757: -18888,
 220261: 343,
 224606: 137,
 229236: 169,
 232031: 3747,
 241683: 1309,
 256429: 28,
 280853: 112,
 1058201: 2366,
 1063276: -793,
 1064571: -167}

In [87]:
total_tsus={}
for i in t_sus_culture.keys():
    if(i in tsus_dict.keys()):
        total_tsus[i]=min(tsus_dict[i],t_sus_culture[i])
    else:
        total_tsus[i]=t_sus_culture[i]

In [88]:
total_tsus

{3155670: -438095,
 2586036: -78866,
 3150261: -45435,
 2596477: -35439,
 3136641: -32186,
 2597831: -28395,
 3138851: -22100,
 2600062: -22090,
 3050144: -21718,
 2590907: -20509,
 3150453: -20069,
 3041223: -19251,
 3138387: -18486,
 2622658: -17279,
 3149711: -16330,
 3132235: -16308,
 3133786: -16227,
 3152037: -15285,
 3114530: -15256,
 3051517: -15195,
 2613719: -14620,
 3158222: -14511,
 2634932: -14457,
 3104707: -13855,
 3147962: -13718,
 3158231: -13678,
 2576928: -13649,
 3155743: -13644,
 3135983: -13588,
 3157289: -13474,
 3069155: -13155,
 3143722: -12998,
 2583429: -12995,
 3145965: -12510,
 3153955: -12061,
 3143759: -12015,
 2595105: -11973,
 3136319: -11769,
 2609875: -11498,
 2612034: -11427,
 3140061: -11401,
 3141817: -11195,
 3139467: -10978,
 3137920: -10973,
 2590822: -10786,
 2634697: -10730,
 3132852: -10564,
 3154495: -10388,
 3143148: -10377,
 3131462: -10365,
 3137480: -10356,
 3135985: -10345,
 3046782: -10309,
 3140706: -10253,
 3051218: -10151,
 2609003:

<h1>---------------------------------------------------------------------------------------</h1>

<h1>For qSOFA</h1>

In [90]:
vitalp=pd.read_csv("vitalPeriodic.csv")
vitalp=vitalp.sort_values(by=['observationoffset'])

In [91]:
vitalp['SOFAscore']=""

In [92]:
vitalp.head()

,vitalperiodicid,patientunitstayid,observationoffset,temperature,sao2,heartrate,respiration,cvp,etco2,systemicsystolic,systemicdiastolic,systemicmean,pasystolic,padiastolic,pamean,st1,st2,st3,icp,SOFAscore
219355,68109528,264762,-3272,NaN,94.0,67.0,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.1,NaN,NaN,
218886,68109488,264762,-3267,NaN,95.0,67.0,29.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,
218934,68109448,264762,-3262,NaN,96.0,67.0,29.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,
218982,68109408,264762,-3257,NaN,96.0,68.0,29.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.1,NaN,NaN,
218998,68109368,264762,-3252,NaN,94.0,68.0,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.1,NaN,NaN,


In [93]:
v_check=np.unique(vitalp['observationoffset'])

In [94]:
for x in v_check:
    if(x<0):
        print(x)

-3272
-3267
-3262
-3257
-3252
-3247
-3242
-3237
-3232
-3227
-3222
-3217
-3212
-3207
-3202
-3197
-3192
-3187
-3182
-3177
-3172
-3167
-3162
-3157
-3152
-3147
-3142
-3137
-3132
-3127
-3122
-3117
-3112
-3107
-3102
-3097
-3092
-3087
-3082
-3077
-3072
-3067
-3062
-3057
-3052
-3047
-3042
-3037
-3032
-3027
-3022
-3017
-3012
-3007
-3002
-2997
-2992
-2987
-2982
-2977
-2972
-2967
-2962
-2957
-2956
-2952
-2951
-2947
-2946
-2942
-2938
-2937
-2933
-2932
-2928
-2927
-2923
-2922
-2918
-2917
-2913
-2912
-2908
-2907
-2903
-2902
-2898
-2897
-2893
-2892
-2888
-2887
-2883
-2882
-2878
-2877
-2873
-2872
-2868
-2867
-2863
-2862
-2858
-2857
-2853
-2852
-2848
-2843
-2838
-2833
-2828
-2823
-2818
-2813
-2808
-2807
-2803
-2802
-2798
-2797
-2793
-2792
-2788
-2787
-2783
-2782
-2778
-2777
-2773
-2772
-2768
-2767
-2763
-2762
-2758
-2757
-2753
-2752
-2748
-2747
-2743
-2742
-2738
-2737
-2733
-2732
-2728
-2727
-2723
-2722
-2718
-2717
-2713
-2712
-2708
-2707
-2703
-2702
-2698
-2697
-2693
-2692
-2688
-2687
-2683
-2682
-267

In [95]:
vitalp['systemicsystolic'].fillna(vitalp['systemicsystolic'].mean(),inplace=True)
vitalp.loc[vitalp['systemicsystolic']>=70,'SOFAscore']=0
vitalp.loc[vitalp['systemicsystolic']<70,'SOFAscore']=1

In [96]:
vitalp.head()

,vitalperiodicid,patientunitstayid,observationoffset,temperature,sao2,heartrate,respiration,cvp,etco2,systemicsystolic,systemicdiastolic,systemicmean,pasystolic,padiastolic,pamean,st1,st2,st3,icp,SOFAscore
219355,68109528,264762,-3272,NaN,94.0,67.0,30.0,NaN,NaN,125.629655,NaN,NaN,NaN,NaN,NaN,NaN,-0.1,NaN,NaN,0
218886,68109488,264762,-3267,NaN,95.0,67.0,29.0,NaN,NaN,125.629655,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0
218934,68109448,264762,-3262,NaN,96.0,67.0,29.0,NaN,NaN,125.629655,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0
218982,68109408,264762,-3257,NaN,96.0,68.0,29.0,NaN,NaN,125.629655,NaN,NaN,NaN,NaN,NaN,NaN,-0.1,NaN,NaN,0
218998,68109368,264762,-3252,NaN,94.0,68.0,30.0,NaN,NaN,125.629655,NaN,NaN,NaN,NaN,NaN,NaN,-0.1,NaN,NaN,0


In [97]:
from collections import defaultdict
qsofa_dict=defaultdict(list)

In [98]:
for i in vitalp.itertuples():
    if(i[10]<=100 and i[7]>=22):
        qsofa_dict[i[2]].append(i[3])

In [99]:
qsofa_dict

defaultdict(list,
            {1069005: [6,
              11,
              16,
              21,
              26,
              41,
              46,
              51,
              56,
              61,
              71,
              76,
              251,
              571,
              576,
              636,
              641,
              646,
              661,
              666,
              671,
              676,
              681,
              686,
              691,
              696,
              701,
              706,
              716,
              746,
              781,
              786,
              791,
              796,
              801,
              811,
              816,
              821,
              826,
              831,
              841,
              846,
              851,
              856,
              861,
              866,
              871,
              876,
              881,
              886,
              891,
              896

In [100]:
def sortlist(listtosort):
    listtosort.sort()
    return listtosort

In [101]:
t_qsofa={}
for key in qsofa_dict:
    sorting_list=qsofa_dict[key]
    sortedlist=sortlist(sorting_list)
    start_off=sortedlist[0]
    if(start_off<=(24*60)):
        t_qsofa[key]=start_off

In [102]:
t_qsofa

{1069005: 6,
 1053280: 6,
 304989: 8,
 238463: 9,
 305890: 24,
 293882: 31,
 987423: 32,
 1071292: 32,
 1033320: 39,
 245536: 45,
 260998: 46,
 1057985: 52,
 979130: 54,
 1058201: 63,
 239049: 65,
 282475: 70,
 236979: 70,
 284517: 74,
 1067864: 74,
 971247: 75,
 968355: 78,
 300435: 86,
 305889: 89,
 1024576: 90,
 246997: 93,
 202312: 99,
 1062433: 100,
 1057145: 107,
 224606: 118,
 211144: 119,
 1073533: 120,
 297791: 128,
 263279: 128,
 276698: 129,
 1066139: 129,
 169525: 133,
 221017: 133,
 261570: 136,
 1073098: 139,
 260132: 143,
 1061219: 144,
 1067893: 147,
 1066133: 157,
 1066190: 159,
 1057727: 160,
 1071720: 161,
 1094153: 169,
 1057192: 170,
 215535: 176,
 253331: 177,
 1013868: 179,
 319808: 181,
 178838: 182,
 302227: 182,
 1056393: 185,
 212922: 190,
 960538: 194,
 1068757: 200,
 1072883: 202,
 271063: 209,
 258915: 210,
 298879: 212,
 143599: 214,
 973583: 215,
 204021: 216,
 294032: 218,
 248438: 220,
 295149: 237,
 1061574: 246,
 1065468: 246,
 292154: 254,
 1071214:

<h2>-------------------------------------------------------------------------------</h2>

In [103]:
respiratory=pd.read_csv("respiratoryCharting.csv")
respiratory.head()

,respchartid,patientunitstayid,respchartoffset,respchartentryoffset,respcharttypecat,respchartvaluelabel,respchartvalue
0,107,184757,2922,2922,respFlowSettings,LPM O2,1
1,407,232032,-24,-24,respFlowSettings,LPM O2,15
2,1549,155956,3254,3254,respFlowSettings,LPM O2,7
3,1729,151226,478,478,respFlowSettings,TV/kg IBW,5.4795
4,1730,151226,478,478,respFlowCareData,Tidal Volume Observed (VT),450


In [104]:
respiratory['respchartvaluelabel'].unique()

array(['LPM O2', 'TV/kg IBW', 'Tidal Volume Observed (VT)', 'PEEP', 'FiO2',
       'Pressure Control', 'Total RR', 'RT Vent On/Off', 'Vent Rate',
       'Tidal Volume (set)', 'Plateau Pressure', 'Pressure Support',
       'Peak Insp. Pressure', 'Endotracheal Position at Lip',
       'Endotracheal Tube Placement', 'Tube Size',
       'Ventilator Heater Temperature', 'Exhaled TV (patient)',
       'Mean Airway Pressure', 'Exhaled MV', 'CPAP', 'RR (patient)',
       'SaO2', 'Head of Bed Elevation', 'Inspiratory Pressure, Set',
       'Flow Sensitivity', 'Tidal Volume, Delivered', 'Humidifier Temp',
       'Inspiratory Flow Rate', 'Minute Volume, Spontaneous', 'PEEP/CPAP',
       'Exhaled TV (machine)', 'Compliance'], dtype=object)

In [105]:
respFiO2=respiratory.loc[respiratory['respchartvaluelabel']=='FiO2']
respFiO2=respFiO2.sort_values(by=['respchartentryoffset'])
respFiO2.head()

,respchartid,patientunitstayid,respchartoffset,respchartentryoffset,respcharttypecat,respchartvaluelabel,respchartvalue
32415,67310838,965856,-10950,-10950,respFlowSettings,FiO2,50
33912,67537031,965856,-11054,-10822,respFlowSettings,FiO2,100
20823,65444733,965856,-10919,-10821,respFlowSettings,FiO2,50
29706,66833789,965856,-11024,-10821,respFlowSettings,FiO2,70
23882,65927004,965856,-11009,-10821,respFlowSettings,FiO2,70


<h1>For GCS's sofa points</h1>

In [106]:
nurseChart=pd.read_csv("nurseCharting.csv")
nurseChart.head()

,nursingchartid,patientunitstayid,nursingchartoffset,nursingchartentryoffset,nursingchartcelltypecat,nursingchartcelltypevallabel,nursingchartcelltypevalname,nursingchartvalue
0,303894086,142493,8,8,Vital Signs,Non-Invasive BP,Non-Invasive BP Systolic,170
1,148760153,142493,342,342,Scores,Pain Score/Goal,Pain Score,3
2,168828181,142493,147,147,Other Vital Signs and Infusions,Arterial Line MAP (mmHg),Value,70
3,261632035,142493,102,102,Other Vital Signs and Infusions,Mental Status Assessment,Value,X
4,277152840,142493,342,342,Other Vital Signs and Infusions,Arterial Line MAP (mmHg),Value,73


In [107]:
gcs=[]
d_gcs=[]
for row in nurseChart.itertuples():
    if((row.nursingchartcelltypecat=="Scores" and row.nursingchartcelltypevallabel=="Glasgow coma score")):
        #gcs.append([row.patientunitstayid,row.nursingchartentryoffset,row.nursingchartcelltypevallabel,int(row.nursingchartvalue)])
        d_gcs.append(row)    
d_gcs=pd.DataFrame(d_gcs)
d_gcs['SOFAscore']=""

In [108]:
d_gcs.nursingchartvalue=pd.to_numeric(d_gcs.nursingchartvalue,errors='coerce')
d_gcs=d_gcs.dropna(subset=['nursingchartvalue'])
d_gcs.nursingchartvalue=d_gcs.nursingchartvalue.astype(int)

In [109]:
len(d_gcs)

69494

In [110]:
d_gcs.head()

,Index,nursingchartid,patientunitstayid,nursingchartoffset,nursingchartentryoffset,nursingchartcelltypecat,nursingchartcelltypevallabel,nursingchartcelltypevalname,nursingchartvalue,SOFAscore
0,84,174878135,142493,102,102,Scores,Glasgow coma score,GCS Total,10,
1,190,174894838,142493,582,582,Scores,Glasgow coma score,GCS Total,15,
2,204,265154559,142493,12,12,Scores,Glasgow coma score,GCS Total,11,
3,242,211010241,142493,342,342,Scores,Glasgow coma score,GCS Total,15,
4,269,229054198,142494,293,293,Scores,Glasgow coma score,GCS Total,15,


In [111]:
d_gcs.loc[d_gcs['nursingchartvalue']>=15,['SOFAscore']]=0
d_gcs.loc[d_gcs['nursingchartvalue']<15,['SOFAscore']]=1
d_gcs.loc[d_gcs['nursingchartvalue']<13,['SOFAscore']]=2
d_gcs.loc[d_gcs['nursingchartvalue']<10,['SOFAscore']]=3
d_gcs.loc[d_gcs['nursingchartvalue']<6,['SOFAscore']]=4

In [112]:
d_gcs=d_gcs.sort_values(by=['nursingchartentryoffset'])
d_gcs.head()

,Index,nursingchartid,patientunitstayid,nursingchartoffset,nursingchartentryoffset,nursingchartcelltypecat,nursingchartcelltypevallabel,nursingchartcelltypevalname,nursingchartvalue,SOFAscore
66414,949674,684090346,1112115,-18547,-18547,Scores,Glasgow coma score,Motor,6,3
66440,949768,684090349,1112115,-18547,-18547,Scores,Glasgow coma score,GCS Total,15,0
66441,949776,684090347,1112115,-18547,-18547,Scores,Glasgow coma score,Verbal,5,4
66443,949784,684090348,1112115,-18547,-18547,Scores,Glasgow coma score,Eyes,4,4
66424,949699,701423825,1112115,-18510,-18510,Scores,Glasgow coma score,Eyes,4,4


<h1>For Cardiovascular Parameter points</h1>

In [113]:
infusiondrug=pd.read_csv("infusionDrug.csv")
infusiondrug.head()

,infusiondrugid,patientunitstayid,infusionoffset,drugname,drugrate,infusionrate,drugamount,volumeoffluid,patientweight
0,2011411,242544,293,Nitroglycerin (mcg/min),15,NaN,NaN,NaN,NaN
1,1973866,242544,439,Nitroglycerin (mcg/min),15,NaN,NaN,NaN,NaN
2,2008274,242544,8,Nitroglycerin (mcg/min),15,NaN,NaN,NaN,NaN
3,2016011,242544,1231,Nitroglycerin (mcg/min),0,NaN,NaN,NaN,NaN
4,1987652,242544,-22,Nitroglycerin (mcg/min),20,NaN,NaN,NaN,NaN


In [114]:
import re
inf_list=[]
for row in infusiondrug.itertuples():
    if("Norepirephrine" in row.drugname):
        inf_list.append(row)
    if("Epinephrine" in row.drugname):
        inf_list.append(row)
    if("Dopamine" in row.drugname):
        inf_list.append(row)
        

In [115]:
inf_cardio=pd.DataFrame(inf_list)
inf_cardio=inf_cardio.drop(['Index'],axis=1)
inf_cardio['SOFAscore']=""

##df.b = pd.to_numeric(df.b, errors='coerce')
#df = df.dropna(subset=['b'])
#df.b = df.b. astype(int)
'''inf_cardio.drugrate=pd.to_numeric(inf_cardio.drugrate,errors='coerce')
inf_cardio=inf_cardio.dropna(subset=['drugrate'])
inf_cardio.drugrate=inf_cardio.drugrate.astype(int)'''


In [116]:
inf_cardio.drugrate=inf_cardio.drugrate.astype(float)
inf_cardio['drugrate'].fillna(inf_cardio['drugrate'].mean(),inplace=True)
#inf_cardio['drugrate'].fillnull(inf_cardio['drugrate'].mean(),inplace=True)

In [117]:
for row in inf_cardio.itertuples():
    if("(mcg/kg/hr)" in row.drugname):
        inf_cardio['drugrate'][row.Index]/=60

In [118]:
inf_grouping=pd.Series([])

In [119]:
inf_cardio
needed=['Dopamine','Epinephrine','Norepinephrine']
for p in inf_cardio.patientunitstayid.unique():
    p_df=inf_cardio.loc[inf_cardio.patientunitstayid==p]
    for off in p_df.infusionoffset.unique():
        p_off_df=p_df.loc[p_df.infusionoffset==off]
        p_off_df.loc[(p_off_df['drugname'].str.contains('Dopamine')) & (p_off_df['drugrate']>=15),['SOFAscore']]=4
        p_off_df.loc[(p_off_df['drugname'].str.contains('Dopamine')) & (p_off_df['drugrate']<15),['SOFAscore']]=3
        p_off_df.loc[(p_off_df['drugname'].str.contains('Dopamine')) & (p_off_df['drugrate']<5),['SOFAscore']]=2
        p_off_df.loc[(p_off_df['drugname'].str.contains('Epinephrine')) & (p_off_df['drugrate']>0.1),['SOFAscore']]=4
        p_off_df.loc[(p_off_df['drugname'].str.contains('Epinephrine')) & (p_off_df['drugrate']<=0.1),['SOFAscore']]=3
        p_off_df.loc[(p_off_df['drugname'].str.contains('Norepinephrine')) & (p_off_df['drugrate']>0.1),['SOFAscore']]=4
        p_off_df.loc[(p_off_df['drugname'].str.contains('Norepinephrine')) & (p_off_df['drugrate']<=0.1),['SOFAscore']]=3
        inf_grouping=inf_grouping.append(p_off_df)

        



C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexes\api.py:107: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  result = result.union(other)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [120]:
inf_grouping=inf_grouping.sort_values(by=['infusionoffset'])
inf_grouping.head()

,0,infusiondrugid,patientunitstayid,infusionoffset,drugname,drugrate,infusionrate,drugamount,volumeoffluid,patientweight,SOFAscore
256,NaN,17440205.0,977664.0,-208.0,Dopamine (mcg/kg/min),5.0,NaN,NaN,NaN,NaN,3.0
235,NaN,17726732.0,971067.0,-126.0,Dopamine (mcg/kg/min),5.0,NaN,NaN,NaN,NaN,3.0
35,NaN,17404916.0,960849.0,-67.0,Dopamine (mcg/kg/min),2.0,NaN,NaN,NaN,NaN,2.0
367,NaN,22261131.0,1024576.0,-40.0,Epinephrine (mcg/min),0.0,0.0,15.0,250.0,NaN,3.0
298,NaN,24063695.0,996940.0,10.0,Dopamine (mcg/kg/min),20.0,37.5,800.0,250.0,100.0,4.0


#We need to convert ml/hr to mcg/kg/min
#for now, we will proceed as it is which will introduce errors in the system<br>
if("(ml/hr)" in row.drugname):

<h1>----------------------------------------------------------------------------------------------</h1>

<h1>For Lab Related SOFA points</h1>

In [121]:
lab=pd.read_csv("lab.csv")
lab['SOFAscore']=""

<h3>I will refactor the next block using .loc instead of itertuples for optimization</h3>

<h4>So basically, I'm changing the column value of SOFA for pao2 to pao2/fio2. After the calculation below, all sofa scores related to pao2 stand for (pao2/fio2)</h4>

In [122]:
for row in lab.itertuples():
    if (row.labname=='paO2'):
        pid=row.patientunitstayid
        toffset=row.labresultoffset
        index=row.Index
        if(any(respFiO2.loc[(respFiO2['patientunitstayid']==pid) & ((respFiO2['respchartentryoffset']<=toffset+40) & (respFiO2['respchartentryoffset']>=toffset-40)),'respchartvalue'].values)):
            result=row.labresult/(int(respFiO2.loc[(respFiO2['patientunitstayid']==pid) & ((respFiO2['respchartentryoffset']<=toffset+40) & (respFiO2['respchartentryoffset']>=toffset-40)),'respchartvalue'].iloc[0])/100)
            if(result>=400):
                lab['SOFAscore'][index]=0
            elif(result<400 and result>=300):
                lab['SOFAscore'][index]=1
            elif(result<300 and result>=200):
                lab['SOFAscore'][index]=2
            elif(result<200 and result>=100):
                lab['SOFAscore'][index]=3
            else:
                lab['SOFAscore'][index]=4
        else:
            lab['SOFAscore'][index]=0
    
    if(row.labname=="platelets x 1000"):
        if(row.labresult>=150):
            index=row.Index
            lab['SOFAscore'][index]=0
        elif(row.labresult<150 and row.labresult>=100):
            index=row.Index
            lab['SOFAscore'][index]=1
        elif(row.labresult<100 and row.labresult>=50):
            index=row.Index
            lab['SOFAscore'][index]=2
        elif(row.labresult<50 and row.labresult>=20):
            index=row.Index
            lab['SOFAscore'][index]=3
        else:
            index=row.Index
            lab['SOFAscore'][index]=4
    if(row.labname=="total bilirubin"):
        if(row.labresult<1.2):
            index=row.Index
            lab['SOFAscore'][index]=0
        elif(row.labresult<=1.9 and row.labresult>=1.2):
            index=row.Index
            lab['SOFAscore'][index]=1
        elif(row.labresult<=5.9 and row.labresult>=2):
            index=row.Index
            lab['SOFAscore'][index]=2
        elif(row.labresult<=11.9 and row.labresult>=6):
            index=row.Index
            lab['SOFAscore'][index]=3
        else:
            index=row.Index
            lab['SOFAscore'][index]=4
    if(row.labname=="urinary creatinine"):
        if(row.labresult<1.2):
            index=row.Index
            lab['SOFAscore'][index]=0
        elif(row.labresult<=1.9 and row.labresult>=1.2):
            index=row.Index
            lab['SOFAscore'][index]=1
        elif(row.labresult<=3.4 and row.labresult>=2):
            index=row.Index
            lab['SOFAscore'][index]=2
        elif(row.labresult<=4.9 and row.labresult>=3.5):
            index=row.Index
            lab['SOFAscore'][index]=3
        else:
            index=row.Index
            lab['SOFAscore'][index]=4
                
            
        

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set o

In [123]:
lab.head()

,labid,patientunitstayid,labresultoffset,labtypeid,labname,labresult,labresulttext,labmeasurenamesystem,labmeasurenameinterface,labresultrevisedoffset,SOFAscore
0,67615679,142493,180,7,paO2,197.0,197,mm Hg,mm Hg,188,0
1,66865075,142493,237,7,paO2,156.0,156,mm Hg,mm Hg,249,0
2,67615678,142493,180,7,Base Deficit,1.0,1,mEq/L,mmol/L,188,
3,66865078,142493,237,7,FiO2,30.0,30,%,%,470,
4,67615682,142493,180,7,FiO2,30.0,30,%,%,470,


In [124]:
lab_processed=[]
SOFA_req=['paO2','platelets x 1000','total bilirubin','urinary creatinine']
for row in lab.itertuples():
    if(row.labname in SOFA_req):
        lab_processed.append(row)

In [125]:
lab_processed=pd.DataFrame(lab_processed)
lab_processed=lab_processed.drop(['Index'],axis=1)
'''lab_nogroup=lab_processed'''

'lab_nogroup=lab_processed'

In [126]:
'''lab_nogroup'''

'lab_nogroup'

In [127]:
lab_grouped=lab_processed.groupby(['patientunitstayid','labresultoffset'])


In [128]:
lab_grouped.first()

labid  labtypeid           labname  \
patientunitstayid labresultoffset                                           
142493             50               54100434          3  platelets x 1000   
                   180              67615679          7              paO2   
                   237              66865075          7              paO2   
                   292              54833226          3  platelets x 1000   
                   667              53675887          3  platelets x 1000   
142494             1033             50785913          3  platelets x 1000   
142495            -238              44924487          1   total bilirubin   
                  -63               56112543          3  platelets x 1000   
                   806              56189772          3  platelets x 1000   
                   2286             54190069          3  platelets x 1000   
                   2821             68039242          7              paO2   
                   2897             67999013          7              paO2   
                   2940             67760221          7              paO2   
                   2970             66911999          7              paO2   
                   2992             51345773          3  platelets x 1000   
                   2994             68241446          7              paO2   
142674            -1986             51090835          3  platelets x 1000   
                  -604              49963526          3  platelets x 1000   
                   670              51625391          3  platelets x 1000   
143599            -284              52731925          3  platelets x 1000   
                   164              67966035          7              paO2   
                   412              55192920          3  platelets x 1000   
                   1846             46520283          1   total bilirubin   
                   3266             54897320          3  platelets x 1000   
                   4734             53442939          3  platelets x 1000   
143603             35               67277233          7              paO2   
                   300              48732280          1   total bilirubin   
                   321              67676726          7              paO2   
                   1735             54778990          3  platelets x 1000   
                   1755             66843425          7              paO2   
...                                      ...        ...               ...   
1133552            1395            263137535          1   total bilirubin   
                   2821            262676201          1   total bilirubin   
                   2933            274272601          7              paO2   
                   4278            271926053          3  platelets x 1000   
                   5007            267020159          3  platelets x 1000   
                   5730            259487174          1   total bilirubin   
1135185            1031            267831983          3  platelets x 1000   
                   2504            268713804          3  platelets x 1000   
                   4055            268286061          3  platelets x 1000   
                   5461            272359192          3  platelets x 1000   
                   6907            266886401          3  platelets x 1000   
                   8363            267914506          3  platelets x 1000   
1135186           -439             269045456          3  platelets x 1000   
                   62              266616172          3  platelets x 1000   
                   413             269220911          3  platelets x 1000   
                   844             271195572          3  platelets x 1000   
                   938             274946050          7              paO2   
                   1104            267059148          3  platelets x 1000   
                   1513            271422666          3  platelets x 1000   
                   1873            

<h4><p>First use group by to categorise the pid and offset</p>
<p>Then if for a unique pid, offset combination, paO2 and FiO2 come together, retain the paO2 and delete the FiO2 value. Else, continue normal execution</p></h4>

In [129]:
'''for n, g in lab_grouped:
    flag=0
    if(any(g.loc[g['labname']=='paO2','labname'].values)):
        if(any(g.loc[g['labname']=='FiO2','labname'].values)):
            flag=1
    if(flag==1):
        dropid=g.loc[g['labname']=='FiO2','labid'].iloc[0]
        #dropindices=lab_nogroup.labid[lab_nogroup['labid']==dropid]
        
        lab_nogroup.drop(lab_nogroup.index[lab_nogroup['labid']==dropid], inplace=True)
    #print(flag)'''

"for n, g in lab_grouped:\n    flag=0\n    if(any(g.loc[g['labname']=='paO2','labname'].values)):\n        if(any(g.loc[g['labname']=='FiO2','labname'].values)):\n            flag=1\n    if(flag==1):\n        dropid=g.loc[g['labname']=='FiO2','labid'].iloc[0]\n        #dropindices=lab_nogroup.labid[lab_nogroup['labid']==dropid]\n        \n        lab_nogroup.drop(lab_nogroup.index[lab_nogroup['labid']==dropid], inplace=True)\n    #print(flag)"

In [130]:
'''lab_grouped=lab_nogroup.groupby(['patientunitstayid','labresultoffset'])'''

"lab_grouped=lab_nogroup.groupby(['patientunitstayid','labresultoffset'])"

<h1>Preparing the net SOFA score from all the different processed tables</h1>

In [131]:
tsofa_dict=defaultdict(list)

for name, group in lab_grouped:
    pid=name[0]
    toffset=name[1]
    
    off=[]
    sofa=0
    
    sofa+=group.loc[:,'SOFAscore'].iloc[0]
    
    if(any(inf_grouping.loc[(inf_grouping['patientunitstayid']==pid) & ((inf_grouping['infusionoffset']<=toffset+40) & (inf_grouping['infusionoffset']>=toffset-40)),'SOFAscore'].values)):
        sliced=inf_grouping.loc[(inf_grouping['patientunitstayid']==pid) & ((inf_grouping['infusionoffset']<=toffset+40) & (inf_grouping['infusionoffset']>=toffset-40)),['drugname','SOFAscore']]
        for row in sliced.itertuples():
            if("Dopamine" in row.drugname):
                sofa+=row.SOFAscore
                break
            elif("Epinephrine" in row.drugname):
                sofa+=row.SOFAscore
                break
            else:
                sofa+=row.SOFAscore
                break
    else:
        if(any(vitalp.loc[(vitalp['patientunitstayid']==pid) & ((vitalp['observationoffset']<=toffset+5) & (vitalp['observationoffset']>=toffset-5)),'SOFAscore'].values)):
            sofa+=vitalp.loc[(vitalp['patientunitstayid']==pid) & ((vitalp['observationoffset']<=toffset+5) & (vitalp['observationoffset']>=toffset-5)),'SOFAscore'].iloc[0]
        else:
            sofa+=0
     
    if(any(d_gcs.loc[(d_gcs['patientunitstayid']==pid) & ((d_gcs['nursingchartentryoffset']<=toffset+40) & (d_gcs['nursingchartentryoffset']<=toffset-40)),'SOFAscore'].values)):
        sofa+=d_gcs.loc[(d_gcs['patientunitstayid']==pid) & ((d_gcs['nursingchartentryoffset']<=toffset+40) & (d_gcs['nursingchartentryoffset']<=toffset-40)),'SOFAscore'].iloc[0]
    else:
        sofa+=0
    
    #off[toffset]=sofa
    tsofa_dict[pid].append([toffset,sofa])        

In [132]:
t_sofa_cpy=tsofa_dict

In [133]:
from operator import itemgetter
for k in t_sofa_cpy:
    tosort=tsofa_dict[k]
    t_sofa_cpy[k]=sorted(tosort,key=itemgetter(0))
    
    

In [134]:
t_sofa_cpy

defaultdict(list,
            {142493: [[50, 2], [180, 2], [237, 2], [292, 3], [667, 3]],
             142494: [[1033, 1]],
             142495: [[-238, 0],
              [-63, 1],
              [806, 2],
              [2286, 2],
              [2821, 0],
              [2897, 0],
              [2940, 0],
              [2970, 0],
              [2992, 3],
              [2994, 0]],
             142674: [[-1986, 0], [-604, 0], [670, 0]],
             143599: [[-284, 0],
              [164, 0],
              [412, 0],
              [1846, 0],
              [3266, 0],
              [4734, 0]],
             143603: [[35, 0],
              [300, 0],
              [321, 2],
              [1735, 3],
              [1755, 3],
              [3170, 3],
              [3474, 4],
              [4603, 3],
              [6055, 3],
              [7496, 3],
              [8946, 3],
              [10370, 3],
              [11840, 3],
              [12576, 4],
              [13250, 3],
              [14696, 3

<h3>Now, if we need to see if our patient is within the 24 hours observation window or now, first check whether the least time offset
is less than 24*60 mins</h3>
<p>then check each cell for timestamp less than 24 hours, and check for a difference of +2</p>
<p>The lower of the timestamp will be the t_sofa which is being ported along with the patientid to the final_tsofa dictionary</p>

In [135]:
final_tsofa={}
for k in t_sofa_cpy:
    if(t_sofa_cpy[k][0][0]>(24*60)):
        continue
    l=t_sofa_cpy[k]
    flag=0
    #print(l,len(l))
    for i in range(len(l)-1):
        for j in range(i+1,len(l)):
            diff=int(l[j][1])-int(l[i][1])
            #print(diff)
            if(diff>=2):
                final_tsofa[k]=l[i][0]
                #print(final_tsofa[k])
                flag=1
                break
        if(flag==1):
            break

In [136]:
final_tsofa

{142495: -238,
 143603: 35,
 145227: 43,
 147176: -248,
 149756: 911,
 150875: -12363,
 151226: -592,
 151900: -265,
 153646: -59,
 153833: -501,
 155956: -292,
 157644: -167,
 159737: -235,
 162502: -1218,
 163413: -234,
 165269: -181,
 167518: -283,
 169974: -227,
 171536: -111,
 171891: -358,
 171973: -160,
 172015: 9306,
 175631: 213,
 176758: -206,
 178838: -293,
 179269: -493,
 184757: -19176,
 186761: -4107,
 188763: 1192,
 188764: -1252,
 189866: 100,
 193119: -1565,
 202312: -233,
 202707: -482,
 204021: -369,
 204935: -2257,
 205504: -168,
 206664: -123,
 211144: -510,
 211974: -3879,
 212922: 1000,
 212923: 1412,
 215535: -330,
 215816: -6681,
 215978: -168,
 217302: -155,
 217835: -1799,
 219100: -306,
 220261: -185,
 220896: -2722,
 221017: 14,
 222133: -252,
 222408: -164,
 223539: -348,
 224606: 37,
 233699: 99,
 236434: -251,
 236979: 388,
 239049: -1898,
 241683: -181,
 243097: -985,
 243249: -304,
 243629: -263,
 244216: -294,
 244745: 17,
 244926: -2302,
 245562: 63,

In [137]:
final_tsepsis={}
for pid in final_tsofa:
    tsofa=final_tsofa[pid]
    if(pid in tsus_dict.keys()):
        if(tsofa>=(tsus_dict[pid]-(24*60)) and tsofa<=(tsus_dict[pid]+(12*60))):
            final_tsepsis[pid]=min(tsofa,tsus_dict[pid])

In [138]:
final_tsepsis

{143603: 35,
 151226: -592,
 151900: -265,
 179269: -493,
 184757: -19176,
 215978: -168,
 220261: -185,
 224606: 37,
 244216: -294,
 256429: -481,
 279513: -759,
 287910: -194,
 977887: -376,
 1057604: -3307,
 1062330: -640,
 1064571: -527,
 1117332: 29}

In [139]:
timeconsumed=time.time()-starttime
print("Time elapsed: {} mins".format(timeconsumed))

NameError: name 'time' is not defined